<a href="https://colab.research.google.com/github/semhoun/omnius/blob/main/nb/FrenchDPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### Definitions


[Model lists](https://huggingface.co/unsloth):
 - unsloth/Meta-Llama-3.1-8B-bnb-4bit ***Llama-3.1 15 trillion tokens model 2x faster!***
 - unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit
 - unsloth/Meta-Llama-3.1-70B-bnb-4bit
 - unsloth/Meta-Llama-3.1-405B-bnb-4bit ***We also uploaded 4bit for 405b!***
 - unsloth/Mistral-Nemo-Base-2407-bnb-4bit ***New Mistral 12b 2x faster!***
 - unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit
 - unsloth/mistral-7b-v0.3-bnb-4bit ***Mistral v3 2x faster!***
 - unsloth/mistral-7b-instruct-v0.3-bnb-4bit
 - unsloth/Phi-3.5-mini-instruct ***Phi-3.5 2x faster!***
 - unsloth/Phi-3-medium-4k-instruct
 - unsloth/gemma-2-9b-bnb-4bit
 - unsloth/gemma-2-27b-bnb-4bit ***Gemma 2x faster!***
 - unsloth/SmolLM3-3B-bnb-4bit

Remember to go to https://huggingface.co/settings/tokens for a token!


In [1]:
from google.colab import userdata

model_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

#source_model = "mistralai/Mistral-Nemo-Instruct-2407"
source_model = "mistralai/Mistral-7B-Instruct-v0.3"
#model_name = "Claire-12B-v1.0"
model_name = "Claire-Dev"
hf_username = "nsemhoun"
hf_token = userdata.get('HuggingFaceToken')

save_local = False
save_hf = True

### Installation

In [2]:
%%capture
import os, re
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

!pip install --upgrade -qqq uv
try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
except: get_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers>=4.55.3"
!pip install --upgrade unsloth-zoo
!pip install --upgrade unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

KeyboardInterrupt: 

### Unsloth

In [3]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = source_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = model_4bit,
    token = hf_token,
)

==((====))==  Unsloth 2025.9.4: Fast Smollm3 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.629 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

HuggingFaceTB/SmolLM3-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
lora_rank = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [
french-orca-dpo-pairs-revised](https://huggingface.co/datasets/jpacifico/french-orca-dpo-pairs-revised).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [6]:
dpo_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(sample):
    #instruction = "You are an AI assistant. You will be given a task. You must generate a correct answer."
    instruction = sample["system"]
    input       = sample["prompt"]
    accepted    = sample["chosen"]
    rejected    = sample["rejected"]

    sample["prompt"]   = dpo_prompt.format(instruction, input, "")
    sample["chosen"]   = accepted + EOS_TOKEN
    sample["rejected"] = rejected + EOS_TOKEN
    return sample
pass

from datasets import load_dataset
dataset = load_dataset("jpacifico/french-orca-dpo-pairs-revised", split = "train")
dataset = dataset.rename_column("question", "prompt")
dataset = dataset.map(formatting_prompts_func,)

In [7]:
import pprint
row = dataset[1]
print('NB Row: ' + str(len(dataset)))
print('INSTRUCTION: ' + '=' * 50)
pprint.pprint(row["prompt"])
print('ACCEPTED: ' + '=' * 50)
pprint.pprint(row["chosen"])
print('REJECTED: ' + '=' * 50)
pprint.pprint(row["rejected"])

NB Row: 12670
INSTRUCTION: ==================================================
('Below is an instruction that describes a task, paired with an input that '
 'provides further context. Write a response that appropriately completes the '
 'request.\n'
 '\n'
 '### Instruction:\n'
 'Vous êtes un assistant IA. Une tâche vous sera confiée. Vous devez générer '
 'une réponse détaillée et longue.\n'
 '\n'
 '### Input:\n'
 'Le restaurant Midsummer House propose une cuisine chinoise de gamme de prix '
 'modérée, noté 3 sur 5 par les clients, situé près de All Bar One.\n'
 '\n'
 '### Response:\n')
ACCEPTED: ==================================================
('Midsummer House est un restaurant chinois au prix modéré, noté 3/5, situé à '
 'proximité du All Bar One.<|im_end|>')
REJECTED: ==================================================
('Bien sûr! Voici une phrase qui décrit toutes les données que vous avez '
 'fournies\xa0:\n'
 '\n'
 '"Midsummer House est un restaurant chinois à prix modéré avec u

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [8]:
# Enable reward modelling stats
from unsloth import PatchDPOTrainer
PatchDPOTrainer()
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        max_steps = 60,
        #num_train_epochs = 1,
        learning_rate = 5e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
    beta = 0.1,
    train_dataset = dataset,
    tokenizer = tokenizer,
    max_length = 1024,
    max_prompt_length = 512,
)

In [9]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060. Max memory = 11.629 GB.
2.969 GB of memory reserved.


In [ ]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,670 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 30,228,480 of 3,105,327,104 (0.97% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-283.572357,-415.581268,1.751672,2.741462,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-518.612732,-463.387054,2.353153,2.695973,No Log,No Log,No Log
3,0.713700,-0.020711,0.018173,0.250000,-0.038884,-493.249451,-554.297974,1.944762,2.674816,No Log,No Log,No Log
4,0.680600,-0.035098,-0.065270,0.500000,0.030172,-441.614319,-574.095337,1.921335,1.817498,No Log,No Log,No Log
5,0.704300,0.009605,0.029960,0.250000,-0.020355,-233.122925,-392.546692,2.469197,2.956113,No Log,No Log,No Log
6,0.703100,-0.002515,0.015979,0.375000,-0.018495,-189.620850,-256.549652,2.896144,2.515579,No Log,No Log,No Log
7,0.668000,0.016181,-0.036737,0.875000,0.052918,-210.928925,-477.998779,1.892577,2.459954,No Log,No Log,No Log


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
# formatting_prompts_func = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([
    dpo_prompt.format(
        "Continue la séquence de fibonnaci", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
        "", # rejected - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, pad_token_id = tokenizer.eos_token_id)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([
    dpo_prompt.format(
        "Continue la séquence de fibonnaci", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
        "", # rejected - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_" + model_name)  # Local saving
tokenizer.save_pretrained("lora_" + model_name)
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_" + model_name,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = model_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    dpo_prompt.format(
        "Quel est la fameuse grande tour à Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
        "", # rejected - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 64, pad_token_id = tokenizer.eos_token_id)

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
if model_4bit:
  # Merge to 4bit
  if save_local: model.save_pretrained_merged(model_name, tokenizer, save_method = "merged_4bit_forced",)
  if save_hf: model.push_to_hub_merged(hf_username + "/" + model_name, tokenizer, save_method = "merged_4bit_forced", token = hf_token)
else:
  # Merge to 16bit
  if save_local: model.save_pretrained_merged(model_name, tokenizer, save_method = "merged_16bit",)
  if save_hf: model.push_to_hub_merged(hf_username + "/" + model_name, tokenizer, save_method = "merged_16bit", token = hf_token)

# Just LoRA adapters
if save_local:
  model.save_pretrained(model_name, save_method = "lora",)
  tokenizer.save_pretrained(model_name, save_method = "lora",)
if save_hf:
  model.push_to_hub(hf_username + "/" + model_name, save_method = "lora", token = hf_token)
  tokenizer.push_to_hub(hf_username + "/" + model_name, save_method = "lora", token = hf_token)


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to local
if save_local:
  model.save_pretrained_gguf(model_name, tokenizer, quantization_method = "q8_0")
  model.save_pretrained_gguf(model_name, tokenizer, quantization_method = "f16")
  model.save_pretrained_gguf(model_name, tokenizer, quantization_method = "q4_k_m")


# Save to multiple GGUF options - much faster if you want multiple!
if save_hf:
    model.push_to_hub_gguf(
        hf_username + "/" + model_name,
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q4_0", "f16"],
        token = hf_token,
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
